In [1]:
from collections import defaultdict
from collections import Counter
import itertools

import re
with open("dictionaries/LIWC2015 Dictionary - Chinese (Simplified)(adjusted).dic",'r') as f:
    text = f.read()
    
ff = [x.split('\t') for x in text.split("%")[1].split('\n')[1:-1]]
key_to_name = defaultdict(lambda:0)
for name in ff:
    if len(name) < 2: continue
    key_to_name[name[0]] = re.sub('\(.+','',name[1])
    
name_to_key = {}
for key in key_to_name:
    name_to_key[key_to_name[key]] = key

with open("dictionaries/Simplified_Chinese_LIWC2015_Dictionary.dic", 'r') as f:
    text = f.read()

ts = text.split('%')[2].split('\n')[1:-1]
key_to_line = defaultdict(lambda:[])
for line in ts:
    line = line.split('\t')
    word = line[0]
    cats = line[1:]
    
    for cat in cats:
        key_to_line[int(cat)] += [word]

for name in key_to_line:
    words = key_to_line[name]
    cc = len(words)
    with open('words/' + str(name) + '.' + str(cc),'w') as f:
        f.write(' '.join(key_to_line[name]))

with open("dictionaries/linear_factors.txt", 'r') as f:
    fact = f.read()

unnested = []
for name in fact.split('\n'):
    unnested.append(name.split('/')[-1])
    
hf = []
for name in fact.split('\n'):
    hf.append(name.split('/')[0])
hf = list(set(hf + ['othergram','persconc']))


key_to_unnested = {}
for name in unnested:
    key_to_unnested[name_to_key[name]] = name

counter = Counter()
for line in [x.split('\t')[1:] for x in ts]:
    pairs = list(itertools.combinations(line, 2))
    for item in pairs:
        lf_items = []
        for its in item:
            if its not in key_to_unnested:
                break
        else:
            lf_items.append(item)

            cats = '|'.join([key_to_unnested[x] for x in item])
            
            counter.update([cats])

total = sum(counter.values())
counts_co = sorted(counter.items(), key=lambda item: item[1], reverse=True)

cumulative_sum = 0
new_data = []

for item in counts_co:
    cumulative_sum += item[1]
    new_data.append(tuple(item[0].split('|')) + (str(item[1]), str(round(cumulative_sum/total,2))))

with open('cooccurance.txt','w') as f:
    for line in new_data:
        f.write(' '.join(line) + '\n')
        

In [2]:
counter = Counter()
line_list = [x.split('\t')[1:] for x in ts]
for line in line_list:
    pairs = list(itertools.combinations(line, 2))
    for item in pairs:
        lf_items = []
        for its in item:
            lf_items.append(item)
            cats = '|'.join([key_to_name[x] for x in item])
            counter.update([cats])

total = sum(counter.values())
counts_co = sorted(counter.items(), key=lambda item: item[1], reverse=True)

cumulative_sum = 0
new_data = []

for item in counts_co:
    cumulative_sum += item[1]
    new_data.append(tuple(item[0].split('|')) + (str(item[1]),))

In [3]:
kc = {}
for key in key_to_name:
    cc = Counter()
    for cat_num in line_list:
        cat_num2 = cat_num.copy()
        if key in cat_num:
            cat_num2.remove(key)
            cc.update(['+A +B ' + x for x in cat_num2])
            
            for key2 in key_to_name:
                if key2 != key and key2 not in cat_num2:
                    cc.update(['+A -B ' + key2])
            
        else:
            cc.update(['-A +B  ' + x for x in cat_num2])
            
            for key2 in key_to_name:
                if key2 != key and key2 not in cat_num2:
                    cc.update(['-A -B ' + key2])
                    
    kc[key] = cc
            

In [4]:
kc = {}

for key in key_to_name:
    kc[key] = {}
    for key2 in key_to_name:
        if key2 != key:
            kc[key][key2] = {"+A+B": 0, "+A-B": 0, "-A+B": 0, "-A-B": 0}

for cat_num in line_list:
    for key in key_to_name:
        cat_num2 = cat_num.copy()
        if key in cat_num2:
            cat_num2.remove(key)
            for key2 in key_to_name:
                if key2 != key:
                    if key2 in cat_num2:
                        kc[key][key2]["+A+B"] += 1
                    else:
                        kc[key][key2]["+A-B"] += 1
        else:
            for key2 in key_to_name:
                if key2 != key:
                    if key2 in cat_num2:
                        kc[key][key2]["-A+B"] += 1
                    else:
                        kc[key][key2]["-A-B"] += 1


In [7]:
from scipy.stats import fisher_exact
from itertools import product

def test_pair(dicts):
    contingency_table = [[dicts['+A+B'], dicts['+A-B']],
                     [dicts['-A+B'], dicts['-A-B']]]

    odds_ratio, p_value = fisher_exact(contingency_table)
    return p_value

for a,b in product(key_to_name.keys(),key_to_name.keys()):
    if a == b: continue
    dicts = kc[a][b]
    print(a,b,test_pair(dicts))

1 2 1.4511011045035139e-66
1 3 1.7155734329957217e-29
1 4 2.071318102794222e-09
1 5 4.588717806565995e-05
1 6 1.135750409147221e-07
1 7 6.21118262031321e-06
1 8 0.002499593549404903
1 19 0.002499593549404903
1 9 1.634225572934212e-36
1 11 4.8537547289935953e-79
1 12 3.76311464145696e-39
1 13 3.545545341160174e-296
1 14 3.608324855575414e-95
1 15 2.1692967027079076e-239
1 17 8.764803327844187e-108
1 18 1.634225572934212e-36
1 131 1.6617940962980765e-15
1 132 2.090700839813825e-164
1 90 6.539754944102444e-41
1 91 2.2792846167704078e-30
1 92 2.6650775857373965e-85
1 133 1.5342921507425392e-08
1 140 2.9000419168646324e-53
1 141 6.539754944102444e-41
1 142 9.196772346871093e-14
1 22 1.462752334757843e-16
1 23 0.0012017712002649777
1 24 4.3499846434567376e-74
1 25 1.5542209433800208e-69
1 30 1.5859223124307037e-50
1 31 1.861138226048717e-20
1 32 3.327483898694567e-29
1 33 0.00033679858251252294
1 34 1.9457802739148524e-11
1 35 1.1170470464392595e-05
1 40 2.18506102326685e-23
1 41 4.753613666

KeyboardInterrupt: 